In [1]:
import openmc
import math
from copy import deepcopy
import openmc.deplete as od

## Depletion module testing

I will use this notebook to understand the functionality of OpenMC's ``deplete`` module

### 1. ``Integrator``
The ``Integrator`` class is where the `integrate()` function that a user would call to run a depletion simulation lives. The `integrate()` function does several things:

1. Gets the initial nuclide concentration (``with self.operator as conc:``)
2. Gets the source rate (power or neutron flux), time step size, and step index the depletion calculation (``for i, (dt, source_rate) in enumerate(self):``)
3. Solves the transport equation (``conc, res = self._get_bos_data_from_operator(i, source_rate, conc)``)
4. Solves the bateman equations (``proc_time, conc_list, res_list = self(conc, res.rates, dt, source_rate, i)``)
5. Saves the results in an HDF5 file.

In the following cells, I will describe how each of these steps are done in detail.

#### 1a. Obtaining initial nuclide concentration

The initial nuclide concentration is simply a pointer to the `operator` attribute of the `Integrator` object. The `operator` attribute is an instance of the `TransportOperator` class. How does this work? Let's investigate.

First, we need to create an `openmc.model` object

In [2]:
model = openmc.Model()

fuel = openmc.Material(name="uo2")
fuel.add_element("U", 1, percent_type="ao", enrichment=4.25)
fuel.add_element("O", 2)
fuel.set_density("g/cc", 10.4)

clad = openmc.Material(name="clad")
clad.add_element("Zr", 1)
clad.set_density("g/cc", 6)

water = openmc.Material(name="water")
water.add_element("O", 1)
water.add_element("H", 2)
water.set_density("g/cc", 1.0)
water.add_s_alpha_beta("c_H_in_H2O")
materials = openmc.Materials([fuel, clad, water])

radii = [0.42, 0.45]
pin_surfaces = [openmc.ZCylinder(r=r) for r in radii]
pin_univ = openmc.model.pin(pin_surfaces, materials)
bound_box = openmc.rectangular_prism(1.24, 1.24, boundary_type="reflective")
root_cell = openmc.Cell(fill=pin_univ, region=bound_box)
geometry = openmc.Geometry([root_cell])

settings = openmc.Settings()
settings.particles = 1000
settings.inactive = 10
settings.batches = 50

fuel.volume = math.pi * radii[0] ** 2


model.settings = settings
model.materials = materials
model.geometry = geometry

Next, we create an `Operator` using this model as well as a test chain file (more on this later)

In [3]:
operator = openmc.deplete.Operator(model, "../openmc/tests/chain_simple.xml")

We also will need to specify a source rate, as well as a number of depletion time steps

In [4]:
power = 174
time_steps = [30] * 6

Now we form the `Integrator` object:

In [5]:
integrator = od.PredictorIntegrator(operator, time_steps, power, timestep_units='d')

Now let's check out the use of `self.operator`. The `conc` pointer gets passed to the `_get_bos_data_from_operator` function in `abc.py`:
```python
    def _get_bos_data_from_operator(self, step_index, source_rate, bos_conc):
        """Get beginning of step concentrations, reaction rates from Operator
        """
        x = deepcopy(bos_conc)
        res = self.operator(x, source_rate)
        self.operator.write_bos_data(step_index + self._i_res)
        return x, res
```

In [6]:
x = deepcopy(integrator.operator)
integrator.operator(x, power)

TypeError: 'Operator' object is not iterable

Hmm, looks like we are getting an error. Looking at the code, it seems like there should be an error, as `conc` is an `Operator` object, and not an array of floats. Let's try doing some debugging.

In [ ]:
import pdb
pdb.set_trace()
integrator.integrate()
## b ../openmc/openmc/deplete/abc.py:833
## c
## p conc
## p self.operator.initial_condition()

--Return--
None
> /tmp/ipykernel_42640/3412417994.py(2)<module>()
      1 import pdb
----> 2 pdb.set_trace()
      3 integrator.integrate()
      4 ## b ../openmc/openmc/deplete/abc.py:833
      5 ## c

ipdb> b ../openmc/openmc/deplete/abc.py:833
Breakpoint 3 at /home/ooblack/projects/openmc/openmc/deplete/abc.py:833
ipdb> c
    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ####

ipdb> p res
OperatorResult(k=1.4647752563947658+/-0.00392421929825487, rates=ReactionRates([[[1.36470751e-09, 0.00000000e+00],
                [4.53572254e-05, 0.00000000e+00],
                [4.64023298e-12, 0.00000000e+00],
                [4.16111100e-10, 0.00000000e+00],
                [2.56826162e-06, 0.00000000e+00],
                [6.94796051e-10, 0.00000000e+00],
                [4.60670818e-09, 9.91964909e-11],
                [2.09538641e-09, 9.66965380e-09],
                [1.72115793e-10, 2.11000691e-11]]]))
ipdb> p type(res)
<class 'openmc.deplete.abc.OperatorResult'>


So it looks like that the line `with self.operator as conc` executes the `__enter__` special function, and instead returns the inital nuclide concentration instead of an `Operator` object as one would assume by reading the code.

#### 1b. Source rate, timestep size, and step index
Similar to 1a, the `Integrator` class has an `__iter__` special function that returns the timesteps and source rates when passed as an argument to `enumerate()`.

#### 1c. Solving the transport equation.
The second code line of the `_get_bos_data_from_operator` function is
```python
res = self.operator(x, source_rate)
```

This instruction executes the `TransportOperator` class's `__call__` special function, which runs a transport simulation. See `Operator.__call__()` in `operator.py` for the specific implementation using the OpenMC transport solver.

The `_get_bos_data_from...` function returns the inital nuclide concentration for that depletion step (`conc`) as well as the results of the transport simulation (`res`). `res` is an object of the `OperatorResult` class.

__This means that in theory, we could build up another `TransportOperator` subclass that has an appropriate method for running the transport solver, obtaining results therefrom, and storing them in an `OperatorResult` object, and we would not need to change any other component.__

#### 1d. Solving the Bateman equations

The step to solve the bateman equations is a single line:

```python
                proc_time, conc_list, res_list = self(conc, res.rates, dt, source_rate, i)
```
This instruction executes `Integrator.__call__()` special function, which is implemented within specific `Integrator` subclasses.

__My preliminary conclusion is that all of the necessary class stucture to separate out the depletion capabilties of OpenMC from its transport solver are in `abc.py`, as long as we can also find a way detangle the import statements.__

### 2. Import statements
_Quick note: moving `TalliedFissionYieldHelper` to `helpers.py` would allow us to remove the `from openmc.lib import` statement in `abc.py`_